In [4]:
import numpy as np
import pandas as pd

In [26]:
rna_df = pd.read_csv("data/society/All RNA subcellular localization data.txt", sep="\t")

/home/jand/.conda/envs/jand_venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
sub_df = rna_df[rna_df['RNA_category'].isin(['antisense RNA', 'lncRNA', 'lincRNA', 'processed_transcript'])]
sub_df = sub_df[sub_df['Species'] == 'Homo sapiens']
sub_df = sub_df.dropna(subset=['Gene_ID'])

In [73]:
rna_df[rna_df["Gene_ID"] == 'NCBI:140828'][['Gene_ID', 'SubCellular_Localization']]

,Gene_ID,SubCellular_Localization
27711,NCBI:140828,Nucleus
81382,NCBI:140828,Nucleus
110199,NCBI:140828,Exosome


In [58]:
# sub_df.value_counts('SubCellular_Localization')
# sub_df.head(10)

cytosolic = ['Cytosol', 'Cytoplasm', 'Membrane', 'Insoluble cytoplasm', 'Ribosome', 'Ribosome-free cytosol', 'Mitochondrion', 'Endoplasmic reticulum', 'Microsome']
nucleus = ['Nucleus', 'Nucleoplasm', 'Chromatin', 'Nucleolus', 'Nuclear', 'Nuclear speckle', 'Paraspeckle', 'Paraspeckles in the nucleus', 'Nuclear periphery', 'Speckle periphery', 'Nuclear(exclusion from nucleoli)', 'Nuclear membrane', 'Perinuclear', 'Periphery of the nucleus']

sub_df['SubCellular_Localization'] = sub_df['SubCellular_Localization'].replace(cytosolic, 'Cytosolic')
sub_df['SubCellular_Localization'] = sub_df['SubCellular_Localization'].replace(nucleus, 'Nucleus')
sub_df = sub_df[sub_df['SubCellular_Localization'].isin(["Nucleus", "Cytosolic"])]
print(sub_df['SubCellular_Localization'].value_counts())

value_count = sub_df[['Gene_ID', 'SubCellular_Localization']].value_counts()

value_count_df = value_count.reset_index()
local = []
for gene_id in sub_df['Gene_ID'].unique():
    nuc_count = value_count[gene_id]['Nucleus'] if 'Nucleus' in value_count[gene_id].index else 0
    cyt_count = value_count[gene_id]['Cytosolic'] if 'Cytosolic' in value_count[gene_id].index else 0
    local.append([gene_id, nuc_count, cyt_count])
local_df = pd.DataFrame(local, columns=['Gene_ID', 'Nucleus', 'Cytosolic'])
local_df.head(10)

Nucleus      1972
Cytosolic     672
Name: SubCellular_Localization, dtype: int64


,Gene_ID,Nucleus,Cytosolic
0,NCBI:100048912,7,4
1,NCBI:100124700,9,3
2,NCBI:100128098,0,1
3,NCBI:100128191,1,3
4,NCBI:100129387,4,1
5,NCBI:100129550,2,1
6,NCBI:100129617,0,1
7,NCBI:100130581,1,1
8,NCBI:100130967,1,1
9,NCBI:100287314,0,1


In [3]:
local_df.to_csv("data/society/Gene_ID_Nucleus_Cytosolic_0315.csv", index=False)

NameError: name 'local_df' is not defined

In [7]:
nuc_sum = 0
mixed = 0
cyt_sum = 0
for i in range(len(local_df)):
    if local_df["Nucleus"][i] / (local_df["Nucleus"][i] + local_df["Cytosolic"][i]) > 0.75:
        nuc_sum += 1
    elif local_df["Cytosolic"][i] / (local_df["Nucleus"][i] + local_df["Cytosolic"][i]) > 0.75:
        cyt_sum += 1
    else:
        mixed += 1
print("Nucleus= %d, Cytosolic= %d, Mixed= %d" % (nuc_sum, cyt_sum, mixed))

Nucleus= 670, Cytosolic= 130, Mixed= 306


In [11]:
local_df = pd.read_csv("data/society/Gene_ID_Nucleus_Cytosolic_0315.csv")
local_df.head(5)

In [8]:
nuc_sum = 0
mixed = 0
cyt_sum = 0
for i in range(len(local_df)):
    if local_df["Nucleus"][i] / (local_df["Nucleus"][i] + local_df["Cytosolic"][i]) > 0.75:
        local_df["SubCellular_Localization>0.75"][i] = "Nucleus"
    elif local_df["Cytosolic"][i] / (local_df["Nucleus"][i] + local_df["Cytosolic"][i]) > 0.75:
        local_df["SubCellular_Localization>0.75"][i] = "Cytosolic"
    else:
        local_df["SubCellular_Localization>0.75"][i] = "Mixed"
print("nuc_sum= %d, cyt_sum= %d, Mixed= %d" % (nuc_sum, cyt_sum, mixed))
local_df.head(5)

nuc_sum= 0, cyt_sum= 0, Mixed= 0


/home/jand/.conda/envs/jand_venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/jand/.conda/envs/jand_venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jand/.conda/envs/jand_venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

,Gene_ID,Nucleus,Cytosolic,SubCellular_Localization>0.75
0,NCBI:100048912,7,4,Mixed
1,NCBI:100124700,9,3,Mixed
2,NCBI:100128098,0,1,Cytosolic
3,NCBI:100128191,1,3,Mixed
4,NCBI:100129387,4,1,Nucleus


In [10]:
local_df.sort_values(by=["Gene_ID"])
local_df.head(5)

,Gene_ID,Nucleus,Cytosolic,SubCellular_Localization>0.75
0,NCBI:100048912,7,4,Mixed
1,NCBI:100124700,9,3,Mixed
2,NCBI:100128098,0,1,Cytosolic
3,NCBI:100128191,1,3,Mixed
4,NCBI:100129387,4,1,Nucleus


In [9]:
local_df.to_csv("data/society/Gene_ID_Nucleus_Cytosolic_075_0315.csv", index=False)

In [67]:
jand_df = pd.read_csv("data/society/Gene_ID_Nucleus_Cytosolic_075_0316.csv")
list_df = pd.read_csv("data/test_data/list.csv")

In [69]:
jand_df = jand_df[jand_df['SubCellular_Localization>0.75'] != 'Mixed'][['Gene_ID', 'Cytosolic', 'Nucleus']]
list_df = list_df[['Gene_ID', 'Cytosolic', 'Nucleus']]

In [85]:
print(jand_df.head(10))
print(list_df.head(10))

800
          Gene_ID  Cytosolic  Nucleus
0  NCBI:100009676          0        2
1  NCBI:100093630          0        3
2  NCBI:100113386          0        1
3  NCBI:100126793          0        2
4  NCBI:100127888          0        2
5  NCBI:100128098          1        0
6  NCBI:100128262          0        1
7  NCBI:100128285          0        2
8  NCBI:100128385          0        1
9  NCBI:100128560          0        1


In [32]:
jand_df.merge(list_df, how='inner', on='Gene_ID')

AttributeError: 'Series' object has no attribute 'merge'

In [81]:
pd.concat([jand_df, list_df, list_df]).drop_duplicates(keep=False)

,Gene_ID,Cytosolic,Nucleus
4,NCBI:100129387,1,4
14,NCBI:100289178,0,4
21,NCBI:100505648,0,4
23,NCBI:100505854,0,3
25,NCBI:100505894,1,5
82,NCBI:105221694,0,4
94,NCBI:10984,1,7
102,NCBI:140828,0,2
116,NCBI:25845,0,2
117,NCBI:283120,2,0


In [79]:
sub_df[sub_df["Gene_ID"] == 'NCBI:90632']

,RNALocate_ID,Gene_ID,Gene_Name,Gene_symbol,RNA_category,Species,PMID,SubCellular_Localization,Description
28091,RLID:02000741,NCBI:90632,LINC00473,LINC00473,lncRNA,Homo sapiens,29353885.0,Nucleus,RNA in situ hybridization analysis demonstrate...
28092,RLID:02000742,NCBI:90632,LINC00473,LINC00473,lncRNA,Homo sapiens,27140397.0,Nucleus,"To validate LINC00473 as a nuclear lncRNA, we ..."
81568,RLID-D:02002136,NCBI:90632,LINC00473,LINC00473,lncRNA,Homo sapiens,NaN,Nucleus,NaN
112990,RLID-D:12006336,NCBI:90632,LINC00473,LINC00473,lncRNA,Homo sapiens,NaN,Exosome,NaN


In [84]:
pd.concat([list_df, jand_df, jand_df]).drop_duplicates(keep=False)

,Gene_ID,Cytosolic,Nucleus
64,NCBI:100289178,0,2
81,NCBI:100505648,0,3
87,NCBI:100505854,0,2
88,NCBI:100505894,1,4
425,NCBI:105221694,0,3
484,NCBI:10984,1,4
506,NCBI:140828,0,1
547,NCBI:25845,0,1
553,NCBI:283120,1,0
594,NCBI:286467,1,4
